In [1]:
from imdb_fasttext import lazy_load_imdb_data, fastText_keras
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import SGD, Adagrad # Adam doesn't currently support autograd with embedding layers

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = lazy_load_imdb_data()
x_train.shape

Lazy load successful


(25000, 400)

In [3]:
fastText_keras(max_features=20000, embedding_dims=50, maxlen=400).summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total params: 1,000,102
Trainable params: 1,000,102
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
______________________________________________

In [4]:
class TorchFastText(nn.Module):
    
    def __init__(self, max_features, embedding_dims, maxlen, num_classes=2):
        super(TorchFastText, self).__init__()
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.maxlen = maxlen
        self.num_classes = num_classes
        
        self.embeds = nn.EmbeddingBag(max_features, embedding_dims, mode='mean')
        self.linear = nn.Linear(self.embedding_dims, self.num_classes)
        
    def forward(self, x):
        embedded_sentence = self.embeds(x)
        z = self.linear(embedded_sentence)
        predicted = nn.functional.softmax(z)
        return predicted

N = 32
max_features = 20000
embedding_dims = 50
maxlen = 400
model = TorchFastText(max_features, embedding_dims, maxlen)    

In [5]:
binary_loss = nn.BCELoss()
optimizer = Adagrad(model.parameters(), lr=0.01)

In [6]:
from torch.utils.data import Dataset, DataLoader

class MyData(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        print(f"x.shape: {self.x.shape}")
        print(f"y.shape: {self.y.shape}")
        
    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, idx):
        y_i = torch.FloatTensor(self.y[idx, :])
        x_i = torch.LongTensor(self.x[idx].tolist())

        return {"x":x_i, "y":y_i}
    

In [7]:
training_data = MyData(x_train, y_train)
testing_data = MyData(x_test, y_test)

training_loader = DataLoader(training_data, batch_size=1)
testing_loader = DataLoader(testing_data, batch_size=1)

x.shape: (25000, 400)
y.shape: (25000, 2)
x.shape: (25000, 400)
y.shape: (25000, 2)


In [8]:
from tqdm import tqdm  # for progress bar
def train(epoch):
    print('-'*10)
    print(f'Epoch: {epoch+1}')
    for batch in tqdm(training_loader):
        # Get the inputs and wrap as Variables
        batch_x = Variable(batch["x"])
        batch_y = Variable(batch["y"])
    
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(batch_x)
        loss = binary_loss(outputs, batch_y)
        loss.backward()
        optimizer.step()

In [9]:
# inspiration: https://github.com/pytorch/examples/blob/master/mnist/main.py
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for batch in tqdm(testing_loader):
        batch["x"], batch["y"] = Variable(batch["x"]), Variable(batch["y"])
        outputs = model(batch["x"])
        test_loss += binary_loss(outputs, batch['y'])
        prediction = outputs.data.max(1, keepdim=True)[1]
        label = batch['y'].data.max(1, keepdim=True)[1]
        correct += prediction.eq(torch.LongTensor(label)).sum()
    
    test_loss /= len(testing_loader.dataset)
    accuracy = 100. * correct / len(testing_loader.dataset)
    print(f'Average Test loss: {test_loss.data[0]}')
    print(f'Accuracy: {accuracy}')


In [10]:
n_epochs = 5

for i in range(n_epochs):
    train(i)
test()

  0%|          | 0/25000 [00:00<?, ?it/s]

----------
Epoch: 1


/home/aarurane/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
  0%|          | 3/25000 [00:00<17:11, 24.24it/s]

----------
Epoch: 2


  0%|          | 3/25000 [00:00<17:35, 23.69it/s]

...

----------
Epoch: 4


  0%|          | 11/25000 [00:00<04:04, 102.06it/s]

----------
Epoch: 5


100%|██████████| 25000/25000 [00:31<00:00, 794.89it/s]


Average Test loss: 0.49951988458633423
Accuracy: 80.572
